In [166]:
begin 
    using StaticArrays, LinearAlgebra
    import Base: +,-,*
end 

In [167]:
#MultiDual struct defined
struct MultiDual{N,T} <: Number
    val::T
    derivs::SVector{N,T}
end 

In [168]:
#Show method for MultiDual Numbers
Base.show(io::IO,x::MultiDual) = print(io,x.val," + ",x.derivs," ε")

In [169]:
begin
    import Base:convert,promote_rule
    Base.convert(::Type{MultiDual{N,T}}, x::T) where {N, T<:Real} = MultiDual(x, zeros(SVector{N,T}));
    Base.promote_rule(::Type{MultiDual{N,T}} , ::Type{T}) where {N,T<:Real}=MultiDual{N,T};
end

In [170]:
b=2

2

In [171]:
a= MultiDual(2,SVector(1,0))

2 + [1, 0] ε

In [193]:
a+b

4 + [1, 0] ε

In [172]:
function Base.:+(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}    # Defining Addition for MultiDual Numbers
    return MultiDual{N,T}(f.val + g.val, f.derivs + g.derivs)
end

In [173]:
function Base.:*(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}    #Defining Multiplication for MultiDual Numbers
        return MultiDual{N,T}(f.val * g.val, f.val .* g.derivs + g.val .* f.derivs)
    end

In [174]:
function Base.:/(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}    #Defining Division for MultiDual Numbers
    return MultiDual{N,Float64}(f.val/g.val, (g.val*f.derivs-f.val*g.derivs)/(g.val*g.val))
end


In [175]:
m=MultiDual(2,SVector(1,0))
n=MultiDual(2,SVector(0,1))
m/n

1.0 + [0.5, -0.5] ε

In [176]:
function Base.:-(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}  #Subtraction
    return MultiDual{N,T}(f.val-g.val, f.derivs-g.derivs)
end


In [177]:
function Base.:sin(f::MultiDual{N,T}) where {N,T}          # sin() function
    return MultiDual(sin(f.val), cos(f.val)*f.derivs)
end

In [178]:
function Base.:cos(f::MultiDual{N,T}) where {N,T}     # cos() function
    return MultiDual(cos(f.val),-sin(f.val)*f.derivs)
end

In [179]:
function Base.:exp(f::MultiDual{N,T}) where {N,T}     # exp() function
    return MultiDual(exp(f.val),exp(f.val)*f.derivs)
end

In [180]:
function Base.:log(f::MultiDual{N,T}) where {N,T}     # log() function
    return MultiDual(log(f.val),(1/(f.val))*f.derivs)
end

In [181]:
function Base.:abs(f::MultiDual{N,T}) where {N,T}    # abs() function
    return MultiDual{N,T}(abs(f.val), f.val>0 ? f.derivs : -f.derivs)
end

In [182]:
function Base.:^(f::MultiDual{N,T}, n::Int64) where {N,T} # ^ function for non integral powers too
    return MultiDual((f.val)^n, (n*(f.val)^(n-1))*f.derivs )
end

In [183]:
#Comparisons between MultiDual and Number

function Base.:<(f::MultiDual{N,T}, n::Number) where {N,T} 
    return f.val<n;
end

function Base.:<(n::Number, f::MultiDual{N,T}) where {N,T}
      return !(f<n);
end

function Base.:>(f::MultiDual{N,T}, n::Number) where {N,T}
    return (n<f);
end

function Base.:>(n::Number, f::MultiDual{N,T}) where {N,T}
    return (f<n);
end



In [184]:
#Comparisons between two MultiDual Numbers

function Base.:<(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return (f.val<g.val)
end


function Base.:>(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return (f.val>g.val)
end



In [185]:
h(x, y) = x + y

h (generic function with 1 method)

In [186]:
grad(f,p,q) = f(MultiDual(p, SVector(one(p),zero(p))), MultiDual(q, SVector(zero(q),one(q)))).derivs

grad (generic function with 1 method)

In [187]:
begin
    x=MultiDual(3, SVector(1,0))
    y=MultiDual(2, SVector(0,1))
    h(x,y)
end

5 + [1, 1] ε

In [188]:
grad((p,q)->p^2*q,3,2)

2-element SVector{2, Int64} with indices SOneTo(2):
 12
  9

In [189]:
p=2
q=MultiDual(4,SVector(1,0))
r=MultiDual(3,SVector(0,1))
q>r

true

In [190]:
#Jacobian for 2 variables

function JacobianMat(F,u,v)
   return [grad((x,y)->F(x,y)[1],u,v) grad((x,y)->F(x,y)[2],u,v)];
end
    
function Jacobian(F,u,v)
    return det(JacobianMat(F,u,v));
end

Jacobian (generic function with 1 method)

In [191]:
X(x,y)=[x^2+y^2 x*y^2]

X (generic function with 1 method)

In [192]:
A=Jacobian(X,2,3)

-6.0